In [136]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

##########################################################################
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "/Users/im-yeongje/python/chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

#개인적 코드(수원 카페 추가)
#gu_list = ['영통구','팔달구','장안구','권선구', '처인구', '수지구', '기흥구', '화성시']
gu_list = ['권선구']

def main():
    global driver, load_wb, review_num, df
    df = pd.DataFrame(columns = ['cafe_name', 'review', 'rating'])

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    for i, place in enumerate(gu_list):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place + '카페')
    df.to_csv('SUWON_GS_cafe_review.csv')

    driver.quit()
    print("finish")

def grade_star(width):
    if 'style="width:100%;"' in width:
        return 5
    elif 'style="width:80%;"' in width:
        return 4
    elif 'style="width:60%;"' in width:
        return 3
    elif 'style="width:40%;"' in width:
        return 2
    elif 'style="width:20%;"' in width:
        return 1
    else:
        return 0
    
def search(place):
    global driver

    search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 페이지 읽기
        for i in range(2, 35):
            if i == 2:
                j = 2
            print(i, j)
            try:
                if i == 6 or i == 11 or i == 16 or i == 21 or i == 26 or i == 31:
                    j = 1
                    driver.find_element(By.XPATH, '//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
                    sleep(1)
            except (NoSuchElementException, ElementNotInteractableException):
                print("no more nect")
                break
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(j) + '"]'
            driver.find_element(By.XPATH, xPath).send_keys(Keys.ENTER)
            sleep(1)
            j += 1

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링과 저장하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    
    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element(By.XPATH, detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        while True:
            try:
                driver.find_element(By.LINK_TEXT, '후기 더보기').send_keys(Keys.ENTER) # 후기 더보기 버튼 끝까지 클릭
                sleep(1)
            except (NoSuchElementException, ElementNotInteractableException):
                extract_review(place_name) # 리뷰 추출
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver, df
    #data = pd.DataFrame(columns = ['cafe_name', 'review', 'rating'])

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = grade_star(str(review.select('.grade_star > span'))) # 별점
            val = ''
            new_data = {'cafe_name': [place_name], 'review': [comment[0].text], 'rating': [rating]}
            new_df = pd.DataFrame(new_data)
            df = pd.concat([df,new_df])
            if len(comment) != 0:
                if len(str(rating)) != 0:
                    val = comment[0].text + '/' + str(rating) + '점'
                else:
                    val = comment[0].text + '/0'
                #print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()


/var/folders/0z/_2xlgtcn1w9gckgd5b2_qxlc0000gn/T/ipykernel_80376/886670304.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


##### 0
#### 이리부농
no review in crawling
#### 마디프
no review in crawling
#### 스타벅스 수원권선DT점
no review in crawling
#### 까루테
no review in crawling
#### 카페 하이라이트
no review in crawling
#### 스타벅스 수원금곡점
no review in crawling
#### 스타벅스 수원세류DT점
no review in crawling
#### 스타벅스 수원권선점
no review in crawling
#### 스타벅스 수원호매실점
no review in crawling
#### 바르바커피 수원본점
no review in crawling
#### 포커스온나우
no review in crawling
#### 카페7
no review in crawling
#### 블레스롤 수원터미널점
no review in crawling
#### 헬로방방 수원점
no review in crawling
#### 오니리크
no review in crawling
2 2
#### 오쿠앙
no review in crawling
#### 1059-3 Coffee House
no review in crawling
#### 투썸플레이스 수원여대점
no review in crawling
#### 스타벅스 수원세류역DT점
no review in crawling
#### 커피인우드
no review in crawling
#### 너라는계절
no review in crawling
#### 투썸플레이스 수원고색점
no review in crawling
#### 투썸플레이스 수원호매실점
no review in crawling
#### 파스쿠찌 수원권선DT점
no review in crawling
#### 할리스 수원권선행복점
no review in crawling
#### 할리스 수원권선점
no review in crawling
#### 탐앤탐스 수원권선24시점
no review in

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: headless chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x000000010280af44 chromedriver + 3813188
1   chromedriver                        0x000000010279cae0 chromedriver + 3361504
2   chromedriver                        0x00000001024a0c5c chromedriver + 232540
3   chromedriver                        0x000000010248f0b0 chromedriver + 159920
4   chromedriver                        0x000000010248ee8c chromedriver + 159372
5   chromedriver                        0x000000010248dfec chromedriver + 155628
6   chromedriver                        0x000000010248e460 chromedriver + 156768
7   chromedriver                        0x0000000102499900 chromedriver + 203008
8   chromedriver                        0x00000001024a7948 chromedriver + 260424
9   chromedriver                        0x000000010248e7a4 chromedriver + 157604
10  chromedriver                        